<a href="https://colab.research.google.com/github/AbdelrahmanMosly/Chess2D-SDL/blob/main/PR02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

|               Name               |            ID          |
|           -------------          |      -------------     |
|         Abdelrahamn Bahaa         |         19015882       |
|         Louai Zahran             |         19016198       |
|         Mohamed Magdy            |         19016479       |

In [133]:
import numpy as np
X=np.array([
    [2,4],     [3, 3],   [3, 4],       [5, 4],
    [5, 6],    [5, 8],    [6, 4],    [6, 5],
    [6 ,7],    [7,3],    [7, 4],    [8, 2],
    [9, 4],    [10, 6],    [10, 7],    [10, 8],
    [11,5],    [11, 8],    [12, 7],    [13, 6],
    [13, 7],    [14, 6],    [15, 4],    [15, 5]
            ])


In [134]:
def rbf_kernel(X,gamma):
    """
    Computes the RBF kernel matrix using the formula:
    K(x_i, x_j) = exp(-gamma * ||x_i - x_j||^2)
    
    Parameters:
    X (ndarray): the data matrix with shape (n_samples, n_features)
    gamma (float): the gamma parameter for the RBF kernel
    
    Returns:
    K (ndarray): the RBF kernel matrix with shape (n_samples, n_samples)
    """
    n_samples=X.shape[0]
    K=np.zeros((n_samples,n_samples))
    for i in range(n_samples):
      for j in range(n_samples):
        diff=X[i]-X[j]
        K[i,j]=np.exp(-gamma*np.dot(diff,diff))
    return K


In [135]:
from sklearn.neighbors import NearestNeighbors
def sim_matrix_using_KNN(X,n_neighbors=3):
  """
  Computes the similarity graph using 3-NN graph, where Sim(xi,xj)=1 
  iff xj is one of the nearest three points to xi .
  
  Parameters:
  X (ndarray): the data matrix with shape (n_samples, n_features)
  n_neighbors(int): number of neighbors to be used in sim matrix
  Returns:
  W (ndarray): the similarity matrix with shape (n_samples, n_samples)
  """

  #note n_neighbors+1 as it include its point so we need to add 1 to get the desired result
  threeNN=NearestNeighbors(n_neighbors=n_neighbors+1).fit(X)
  distances,indices=threeNN.kneighbors(X)
  n_samples=X.shape[0]
  W=np.zeros((n_samples,n_samples))
  # print(distances)
  # print(indices)

  #i is the sample 
  #j is index for the nearest neighbor for i
  for i in range(n_samples):  
    for j in indices[i]:
        if j != i: #dont use one with it self
            W[i, j] = 1
            W[j, i] = 1
  return W

1. Compute the similarity matrix W e
2. normalized cut then B <-—  La
3. La=D^-1 . W
4. get B eigen vector and choose smallest eigen values correspond to this eigen vector 
5. U(n ->n-k+1)
6. Y normalize row U
7. cluster via k-means on Y

In [136]:
from sklearn.cluster import KMeans
def k_way_normalized_cut(X, k, gamma=None):
    """
    Performs k-way normalized cut clustering on a dataset using either an RBF kernel or a K-nearest neighbors graph.

    Parameters:
        X (np.ndarray): A numpy array of shape (n_samples, n_features) representing the dataset to be clustered.
        k (int): An integer representing the number of clusters to create.
        gamma (float, optional): A float representing the gamma parameter for the RBF kernel, or None to use a KNN graph instead. Default is None.

    Returns:
        tuple: A tuple containing the cluster assignments, reduced eigenspace matrix, trained KMeans model, and value of gamma used (if applicable). The first element is a numpy array of shape (n_samples,) containing the cluster assignments for each data point. The second element is a numpy array of shape (n_samples, k) containing the reduced eigenspace matrix. The third element is a KMeans object representing the trained KMeans model. The fourth element is a float representing the value of the gamma parameter used, or None if a KNN graph was used instead.
    """
    if gamma is not None:
        K = rbf_kernel(X, gamma)
        W = K
    else:
        W = sim_matrix_using_KNN(X)

    D = np.diag(np.sum(W, axis=1))
  
    L = D - W 

    La = np.dot(np.linalg.inv(D),L)

    values, Y = np.linalg.eig(La)
  
    asc_sorted_indices = np.argsort(values)
    values=values[asc_sorted_indices]

    Y=Y[:,asc_sorted_indices]

    Y = Y[:, 0:k]

    Y= Y/np.linalg.norm(Y,axis=1,keepdims=1)

    kmeans = KMeans(n_clusters=k,n_init=10).fit(Y)
    labels = kmeans.labels_
    return labels, Y ,kmeans ,gamma

k_way_normalized_cut_output=k_way_normalized_cut(X,3,0.1)
print(k_way_normalized_cut_output[0])

[1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 0 0 0 0 0]


In [137]:
def rbf_kernel_point_and_dataset(X,point,gamma):
    """
    Computes the radial basis function (RBF) kernel between a given data point and a dataset.

    Parameters:
        X (np.ndarray): A numpy array of shape (n_samples, n_features) representing a dataset.
        point (np.ndarray): A numpy array of shape (n_features,) representing a single data point.
        gamma (float): A parameter for the RBF kernel function. Higher values of gamma will result in a more complex decision function, as it inversly propotional with bandwidth
         i.e., the model will be more likely to overfit. Lower values of gamma will result in a simpler decision function, i.e., the model will be more likely to underfit.

    Returns:
        np.ndarray: A 1D numpy array of shape (n_samples,) representing the RBF kernel between `point` and each sample in `X`.
    """
    n_samples=X.shape[0]
    K=np.zeros(n_samples)

    for j in range(n_samples):
      diff=point-X[j]
      K[j]=np.exp(-gamma*np.dot(diff,diff))

    return K


In [138]:
def sim_using_KNN_point_and_dataset(X,point,n_neighbors=3):
  """
    Computes the similarity between a given data point and a dataset using K-nearest neighbors.

    Parameters:
        X (np.ndarray): A numpy array of shape (n_samples, n_features) representing a dataset.
        point (np.ndarray): A numpy array of shape (n_features,) representing a single data point.
        n_neighbors (int): An integer that specifies the number of nearest neighbors to consider when computing the similarity between `point` and `X`. Default is 3.

    Returns:
        np.ndarray: A 1D numpy array of shape (n_samples,) representing the similarity of Each element of the array is either 0 or 1, where 1 indicates that the corresponding sample in `X` is one of the `n_neighbors` nearest neighbors of `point`.
  """
  #note n_neighbors as now the point supposedly not in the original data so we check the neighbors in the data 
  threeNN=NearestNeighbors(n_neighbors=n_neighbors).fit(X)
  n_samples=X.shape[0]
  W=np.zeros(n_samples)
  point2D=np.zeros((1,point.shape[0]))
  point2D[0]=point
  _,indices=threeNN.kneighbors(point2D)
  for index in indices:
      W[index] = 1
  return W

In [139]:
def predict_new_point(new_point, k_way_normalized_cut_output):
    """
    Given a new data point and the output of a trained k-way normalized cut model, predicts the cluster assignment of the new point.

    Parameters:
        new_point (np.ndarray): A numpy array of shape (n_features,) representing a single data point to be classified.

        k_way_normalized_cut_output (tuple): A tuple containing the necessary outputs from a trained k-way normalized cut model.
         The tuple should contain the original dataset, the reduced eigenspace matrix Y_train, the KMeans clustering model kmeans_model,
          and the gamma parameter used to compute the RBF kernel, if applicable.

    Returns:
        int: An integer representing the predicted cluster assignment of the new point.
    """
    # Unpack the necessary outputs from the trained k_way_normalized_cut function
    Y_train = k_way_normalized_cut_output[1]
    kmeans_model=k_way_normalized_cut_output[2]
    gamma=k_way_normalized_cut_output[3]
    
    # Compute similarity between the new point and all the data points in the original dataset
    if gamma is not None:
      similarities = rbf_kernel_point_and_dataset(X,new_point, gamma)
    else:
      similarities = sim_using_KNN_point_and_dataset(X,new_point)   
    # Normalize vector
    similarities = similarities / np.linalg.norm(similarities)

    # Project the normalized row vector onto the reduced eigenspace
    reduced_coords = np.dot(similarities, Y_train)
    
    # Predict the cluster assignment for the new point based on its coordinates in the reduced eigenspace
    label = kmeans_model.predict(reduced_coords.reshape(1, -1))[0]
    
    return label


In [140]:
def test_prediction():
  predicted=[]

  for point in X:
   predicted.append(predict_new_point(point,k_way_normalized_cut_output))

  print(np.allclose(predicted,k_way_normalized_cut_output[0]))
test_prediction()

True
